In [84]:
#%%
import pandas as pd
import numpy as np
import xlwings as xw
import os
from pathlib import Path
import secret

In [ ]:
df = pd.DataFrame(data=np.random.rand(100, 10),
                  columns=[f"Trial {i}" for i in range(1,11)])
df

In [85]:
file = 'J11998 MODEC - MV Series FPSO - COM R3.xlsm'
downloads = '/Users/infowizard/Downloads/'
file_path = Path(downloads, file)
wb = xw.Book(file_path, password='BP85')

In [93]:
visible_sheets = [sht.name for sht in wb.sheets if sht.visible]

In [86]:
# Read and set currency from FX sheet
fx = wb.sheets['FX']
exchange_rates = dict(fx.range('A2:B9').value)
exchange_rates

{'AUD': 1.03,
 'EUR': 1.64,
 'GBP': 1.82,
 'JPY': 0.0126,
 'NOK': 0.1687,
 'DKK': 0.2061,
 'SGD': 1.0,
 'USD': 1.41}

In [87]:
# Currency to quote
quoted_currency = fx.range('B12').value
quoted_currency

'USD'

In [88]:
# Get project info
project_info = dict(fx.range('A36:B45').value)
project_info

{'Attend to: ': 'Phoebe Yang',
 'Designation: ': 'Procurement Specialist',
 'Client Name: ': 'MODEC Offshore Production Systems (Singapore) Pte Ltd',
 'Client RFQ No: ': '-',
 'Ref Doc No: ': '-',
 'Project Name: ': 'MV SERIES - TELECOMMUNICATION SYSTEM FRAMEWORK',
 'Prepared By: ': 'Thiha',
 'Sales Manager: ': 'Shaun Teo',
 'Jason Ref: ': 'J11998',
 'Revision Num: ': '3'}

In [7]:
paga = wb.sheets['PAGA']
paga.range('D2').value = project_info['Jason Ref: ']

In [96]:
# Read system sheets
cols = ['No', 'Qty', 'Unit', 'Description', 'Unit Price', 'Subtotal', 'Part', 'Cur', 'Cost', 'SubCost', 'Discount']
systems = pd.DataFrame()
defaults = {}
skip_sheets = ['FX', 'Cover', 'Intro', 'ES', 'T&C']
for sheet in visible_sheets:
    if sheet not in skip_sheets:
        ws = wb.sheets[sheet]
        escalation = dict(ws.range('K2:L5').value)
        default_mu = ws.range('H5')
        escalation['default_mu'] = default_mu
        defaults[sheet] = escalation
        last_row = ws.range('D100000').end('up').row  #Returns a number
        last_cell = 'K' + str(last_row) 
        data = ws.range('A8:' + last_cell).options(pd.DataFrame, index=False).value
        data.columns = cols
        data['System'] = str(sheet)
        data['Category'] = 'Product'
        systems = pd.concat([systems, data], join='outer')

In [97]:
xw.view(systems)

In [113]:
# Read Engineering Services
es_cols = ['No', 'Qty', 'Unit', 'Description', 'Unit Price', 'Subtotal', 'Part', 'Cur', 'Cost', 'Man-days', 'Subcost']
es = wb.sheets['ES']
es_last_row = es.range('D100000').end('up').row
es_last_cell = 'K' + str(es_last_row)
eng_service = es.range('A8:' + es_last_cell).options(pd.DataFrame, index=False).value
eng_service.columns = es_cols

In [114]:
xw.view(eng_service)

In [111]:
es_last_cell

'K107'